In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
# print(bostonDF.shape)
# bostonDF.head()

In [ ]:
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):
    
    # 데이터 건수
    N = target_sgd.shape[0]
    # 예측 값. 
    predicted_sgd = w1 * rm_sgd + w2*lstat_sgd + bias
    # 실제값과 예측값의 차이 
    diff_sgd = target_sgd - predicted_sgd
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_sgd.T, diff_sgd))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_sgd.T, diff_sgd))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_sgd))
    
    
    return bias_update, w1_update, w2_update
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    
    for i in range(iter_epochs):
        stochastic_index = np.random.choice(target.shape[0], 1)
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, verbose=True)

In [ ]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    N = target_batch.shape[0]
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    diff_batch = target_batch - predicted_batch
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    

    return bias_update, w1_update, w2_update

def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):

    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    for i in range(iter_epochs):
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

In [ ]:
batch_indexes = np.random.choice(506, 30)
print(batch_indexes)

bostonDF['RM'].values[batch_indexes]


In [ ]:

def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    

    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    

    for i in range(iter_epochs):
        for batch_step in range(0, target.shape[0], batch_size):
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
        
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
        
            if verbose:
                print('Epoch:', i+1,'/', iter_epochs, 'batch step:', batch_step)
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias